In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import create_ribasim_lumping_network

import pandas as pd
import geopandas as gpd

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", UserWarning)

import networkx as nx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
dhydro_dir = Path(base_dir, "d-hydro")
results_dir = Path(base_dir, "results")
network_name = "tki_zutphen_berkel"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden", f"{network_name}_afw_eenheden.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [ ]:
# Create networkanalysis
network = create_ribasim_lumping_network(
    name=network_name, 
    dhydro_dir=dhydro_dir,
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)
# network.export_to_geopackage()

In [ ]:
# Add simulation sets for winter, summer, etc.
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=dhydro_dir,
    simulations_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulations_output_dir="FlowFM\\output",
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="6D"),
);

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data()
# network.export_to_geopackage()

De nieuwe functie - inlezen vanuit de input bestanden

In [ ]:

# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_network_full_data()
# network.export_to_geopackage()

In [ ]:
network.pumps_gdf

In [ ]:
network.nodes_gdf.plot()

In [ ]:
network.edges_gdf

In [ ]:
network.branches_gdf

TEST branches op chainage opknippen (nodig omdat de structures op basis van branch id en chainage in de data zitten)

In [ ]:
# ik kan online geen functies vinden die op basis van chainage langs een lijn kunnen werken. 
# wel vond ik een voorbeeld van een lijn die opgeknipt wordt op een bepaalde afstand. 
# Wellicht kunnen we die gebruiken en dan de branch opknippen op de afstand uit de chainage. 
# En dan van die afgeknipte lijn het eindpunt nemen om tot het punt (de structure op die chainage) te komen. 
# voorbeeld script dat ik tegenkwam op https://www.linkedin.com/pulse/splitting-polyline-eq-dist-segments-python-chonghua-yin/

from shapely.geometry import LineString, Point, Polygon, MultiLineString
def cut(line, distance, lines):
    # Cuts a line in several segments at a distance from its starting point
    if distance <= 0.0 or distance >= line.length:
            return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
                return [
                    LineString(coords[:i+1]),
                    LineString(coords[i:])
                ]
        if pd > distance:
                cp = line.interpolate(distance)
                lines.append(LineString(coords[:i] + [(cp.x, cp.y)]))
                line = LineString([(cp.x, cp.y)] + coords[i:])
                if line.length > distance:
                    cut(line, distance, lines)
                else:
                    lines.append(LineString([(cp.x, cp.y)] + coords[i:]))
                return lines

In [ ]:
test_lijn = network.branches_gdf['geometry'][0]
test_lijn.length

In [ ]:
test_lijn_cut = cut(test_lijn, 10.0, list())[0]
test_lijn_cut

In [ ]:
test_lijn_cut.length

verder met splitnodes en ribasim

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(weirs=True);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    edges=False,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
        'kdu_DR80740070', # inlaat twentekanaal
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
        'kst_ST80830045', # weir tussen uitstroompunten Zutphen
    ], 
    edge_ids_to_include=[],
    node_ids_to_include=[
        1784, 2542 # splitsing afleidingskanaal
    ],
    node_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'culvert': 'TabulatedRatingCurve', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve',
    'edge': 'ManningResistance',
}
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = {
    'sto_AE80770024': 'ManningResistance', 
    'kdu_DR80740070': 'ManningResistance',
    # duikers voor wijk Zuidwijken
    'kdu_DR80940046': 'ManningResistance', 
    'kdu_DR80950043': 'ManningResistance', 
    'kdu_DR80950151': 'ManningResistance', 
}

In [ ]:
# Generate Ribasim model en exporteer naar geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion, 
    split_node_id_conversion=split_node_id_conversion
)
ribasim_model.write(f"{results_dir}/{network.name}")